<div style="text-align:center;">
  <img src="images/molssi_main_horizontal.png" style="display: block; margin: 0 auto; max-height:200px;">
</div>

# Building a Database with SQL and SQLite

<strong>Author(s):</strong> Jessica A. Nash, The Molecular Sciences Software Institute

<div class="alert alert-block alert-info"> 
<h2>Overview</h2>

<strong>Questions:</strong>

* How do we create a database using SQL commands?

* What are SQL commands, and how do we use them to define and manipulate tables?

* How do we interact with a SQLite database using Python?

<strong>Objectives:</strong>

* Understand how to create and initialize a SQLite database.

* Learn the basic SQL commands for creating tables and inserting data.

* Use Python to interact with SQLite databases.

</div>

## Introduction to SQLite

Relational databases are often implemented in some flavor of SQL (Structured Query Language).
Structured Query Language (SQL) is a language used to manage and manipulate relational databases.
Some common SQL databases include MySQL, PostgreSQL, and SQLite.
Each of these has their own SQL dialect, though they share many common features.

In this lesson, we will be learning about SQL command using SQLite. 

SQLite is a lightweight, self-contained SQL database engine that is widely used for local databases in applications. 
It doesn't require a separate server to run, which makes it ideal for use in smaller applications or prototyping.

In this lesson, we'll create a database using SQLite, define our tables, and insert some initial data. 
Python has [built-in support as part of the Python Standard Library to interact with SQLite databases](https://docs.python.org/3/library/sqlite3.html), 
so we'll use the `sqlite3` module to interact with our database. 
At the end of this lesson, we'll also show how you could have performed the same tasks using the SQLite command line interface.

<div class="alert alert-block alert-success">

<h3>Tip: Viewing Database Files</h3>

If you would like to be able to view the database file that is created, we recommend installing [this plugin for VS Code](https://marketplace.visualstudio.com/items?itemName=qwtel.sqlite-viewer).

</div>

## Using SQLite from Python

To use SQLite from Python, we need to import the `sqlite3` module.
We then, create a database connection by giving a file path to the database file we would like to use.
As stated earlier, SQLite is a serverless database, so the database file is the database itself.
In the next cell, we create a sqlite database called `research_articles.db`.

In [ ]:
import os

def remove_db():
    # remove the database file if it exists
    if os.path.exists("research_articles.db"):
        os.remove("research_articles.db")

remove_db()

In [ ]:
import sqlite3

# Connect to a database (or create one if it doesn't exist)
connection = sqlite3.connect('research_articles.db')

print("Opened database successfully")

If you are watching your file system, you will now see that a file has been created in the current directory with the name `research_articles.db`.
If you now view your database file using the SQLite plugin for VS Code, you will see that the database is empty.

After creating the connection, we create a cursor object to interact with the database.

In [ ]:
# Create a cursor object using the connection
cursor = connection.cursor()

# Print a confirmation message
print("Cursor created.")

We will use this cursor to add data to our database.

Currently, we have a database, but the database has no tables or structure.
In order to add data to our database, we first need to define the database structure by defining our databases tables.
As a reminder, we decided to have three tables: `authors`, `articles`, and `author_article`.

To define these, we will have to write SQL commands.
SQL is what is called a "declarative language". 
This means when you use SQL, you are telling the computer *what* to do, but not necessarily *how* it is done. 
You will not control the program flow with loops and conditionals like in Python or C++, rather you issue commands like `SELECT` or `CREATE`.
A SQL command is generally composed of several key parts: keywords, identifiers, operators, and the data itself. 
Keywords are specific terms like `SELECT`, `CREATE`, `INSERT`, and `DELETE`, each referring to a possible action in the database. 

For example, to create a table, you would use the `CREATE TABLE` keyword followed by your chosen table name and details about the columns it should contain.
You use the `CREATE TABLE` keyword followed by the table name and a list of columns with their names and types.

```sql
CREATE TABLE table_name (
    column1_name column1_type conditions,
    column2_name column2_type conditions,
    ...
);
```

For example, to create our `DOI` column in our article table, the syntax would be something like

```
CREATE TABLE IF NOT EXISTS Article (
    DOI TEXT PRIMARY KEY
)
```

This says to create a table called `Article` and to add a column called `DOI`
that has the data type `TEXT` and that this column should be the primary key.

The next cell contains the full SQL command for creating the `article` table.
We first define the SQL command as a multiline string in Python.
Next, we use the cursor object to execute the SQL command.
Finally, we commit the changes to the database.

In [ ]:
# SQL command to create Article table
article_table_command = """
CREATE TABLE IF NOT EXISTS articles (
    DOI TEXT PRIMARY KEY,
    article_title TEXT NOT NULL,
    publication_year INTEGER NOT NULL,
    abstract TEXT
);
"""

# Execute the SQL commands
cursor.execute(article_table_command)

# Commit the changes
connection.commit()

If we now examine our database file using the SQLite plugin for VS Code, we will see that the `article` table has been created.
The primary key (the `DOI` column) is shown with a key icon next to it.
The viewer shows us that we don't have any data in the database yet.

<div style="text-align:center;">
  <img src="images/article_table.png" style="display: block; margin: 0 auto; max-height:400px;">
</div>

## Database Transactions

What we just did in the previous cell is an example of a database **transaction**.
A transaction is a single unit of work that ensures that the change we wanted to make (in this case, creating a table) either happens completely or doesn't happen at all if something goes wrong.

In the cell above, the transaction concludes with the  `commit` method. 
When you execute the `CREATE TABLE` statement, a transaction begins. 
The COMMIT command finalizes this transaction, making all changes permanent and visible to other users or processes interacting with the database. 

In [ ]:
# SQL command to create Article table
author_table_command = """
CREATE TABLE IF NOT EXISTS authors (
    author_id INTEGER PRIMARY KEY AUTOINCREMENT,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    affiliation TEXT
);
"""

# Execute the SQL commands
cursor.execute(author_table_command)

# Commit the changes
connection.commit()


To add the `article_author` table, we need use the concepts of a **foreign key**.
Foreign keys tell SQL that the values in this table will reference another table.
This article-authors table will also have what is called a **composite primary key**.
A composite primary key is a primary key that consists of more than one column.
Usually, a primary key in a table must be unique. In the case of a composite primary key, 
the combination of values for the key must be unique.


In [ ]:
article_author_table_command = """
CREATE TABLE IF NOT EXISTS article_authors (
    article_id TEXT,
    author_id INTEGER,
    PRIMARY KEY (article_id, author_id),
    FOREIGN KEY (article_id) REFERENCES articles(DOI),
    FOREIGN KEY (author_id) REFERENCES authors(author_id)
);
"""

# Execute the SQL commands
cursor.execute(article_author_table_command)

# Commit the changes
connection.commit()


<div class="alert alert-block alert-warning">

## Exercise
Using the examples above, construct SQL queries for creating the `keywords` table and for creating the `article_keyword` associative table.

</div>

# Inserting Data

Let's insert some data into our database.
In the `data` folder of this repository, we've saved data for a single paper (`data/one_paper.json`).
This is an arbitrary paper that was retrieved while we were wrting the tutorial from ChemRxiv.

<div class="alert alert-block alert-success">

### Where did we get this data?

We retrieved this data using the REST API for ChemRxiv and saved it as a json.
In the example below, we pull the most recent article in the "Theoretical and Computational Chemistry" category from ChemRxiv.
Note that this will likely be different when run on different dates.

<div class="alert alert-block">

```python
import requests

import datetime

# most recent theoretical chemistry paper on ChemRXiv
paper = requests.get("https://chemrxiv.org/engage/chemrxiv/public-api/v1/items?categoryIds=605c72ef153207001f6470ce&limit=1")
print(paper.json()["itemHits"])
```
</div>

</div>

In the cell below, we use the Python `json` library to load this previously saved data.


In [ ]:
import json

with open("data/one_paper.json") as f:
    paper = json.load(f)

print(paper)

We can gather data needed for our database from the JSON.
Note that ChemRxiv has a lot more information than we are going to store in our datbase.
In fact, they probably have their own database with this information!
Some choices we've made here are for simplicity and to illustrate the concepts we are learning.

In [ ]:
import datetime

recent_paper ={
    "DOI": paper["itemHits"][0]["item"]["doi"],
    "title": paper["itemHits"][0]["item"]["title"],
    # get the current year - making some assumptions here :)
    "year": datetime.datetime.now().year,
    "abstract": paper["itemHits"][0]["item"]["abstract"],
    "keywords": paper["itemHits"][0]["item"]["keywords"],
    "authors": paper["itemHits"][0]["item"]["authors"]
}

for k,v in recent_paper.items():
    print(f"{k}: {v}")

To insert data into our database, we use the `INSERT INTO` SQL command.
We leave question marks `?` in the SQL command where we want to insert data,
then feed the data to be inserted into the database as a tuple as the second argument to the `execute` method.
This is a safety feature to prevent SQL injection attacks.

In [ ]:
# Insert the data
insert_article_command = """
 INSERT INTO articles (DOI, article_title, publication_year, abstract)
    VALUES (?, ?, ?, ?)"""

cursor.execute(insert_article_command, (recent_paper["DOI"], recent_paper["title"], recent_paper["year"], recent_paper["abstract"]))

# Commit the changes
connection.commit()


Processing our keywords and authors is a bit more complicated.
We need to insert the keywords into the `keywords` table and the authors into the `authors` table, 
then make our associations.
We will first need to add the keywords (or authors) so that the values can get IDs.
Then, using the IDs of the keywords, we can associated the keywords with the paper in the database.

In [ ]:
for keyword in recent_paper["keywords"]:
    print(keyword)

insert_keyword_command = """
    INSERT INTO keywords (keyword)
    VALUES (?)
"""
keyword_ids = []
for keyword in recent_paper["keywords"]:
    cursor.execute(insert_keyword_command, (keyword,))

connection.commit()

Now, we need to add the association between these keywords and the paper we've retrieved.

In order to associate the keywords with our paper, we must first retrieve the keyword ID, then add a new entry in our associative table using that ID and the paper ID.

In [ ]:
paper_id = recent_paper["DOI"]

query_statement = """SELECT keyword_id from keywords WHERE keywords.keyword == ?"""

associative_statement = """INSERT INTO article_keywords (article_id, keyword_id) VALUES (?, ?)"""

for keyword in recent_paper["keywords"]:
    cursor.execute(query_statement, (keyword,) )
    result = cursor.fetchone()[0]
    cursor.execute(associative_statement, (paper_id, result))
    

<div class="alert alert-block alert-success">

<h3>Tip: Using INSERT OR IGNORE to Prevent Duplicate Entries</h3>

In some cases, you might want to insert a keyword only if it doesn't already exist in the table.
SQLite provides a convenient way to handle this with the `INSERT OR IGNORE` statement.
This command attempts to insert the data, but if a conflict (like a `UNIQUE` constraint violation) occurs,
it ignores the insertion instead of throwing an error. 
This approach can help maintain data integrity without needing to manually check for existing entries.

<div class="alert alert-block">

```python

insert_keyword_command = """
    INSERT OR IGNORE INTO keywords (keyword)
    VALUES (?);
"""
```
</div>

</div>




<div class="alert alert-block alert-warning">

## Exercise
Add authors to the database for the paper.
</div>

This notebook has introduced creating and interacting with a database using SQLite and Python.
Using this method, you write "raw" SQL queries and use Python to execute them. 
If you wanted to switch your database to another dialect of SQL such as Postgres or MySQL, you would need to rewrite these queries.

In the next notebook, we'll introduce the concept of an object-relational mapper or ORM. ORMs are programming objects (like classes) that are used to interact with and create databases. 
The offer the advantage of being more secure and transferable across SQL dialects. 